---
**License**

 funnel_mcmc

 Thu Feb 07 16:49:00 2023\
 Copyright  2023\
 Sandro Dias Pinto Vitenti  <vitenti@uel.br>

---
---

 funnel_mcmc\
 Copyright (C) 2023 Sandro Dias Pinto Vitenti  <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [ ]:
import sys

from numcosmo_py import Ncm

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import numpy as np
import math

import getdist
from getdist import plots

from numcosmo_py.experiments.funnel import run_funnel_mcmc

from numcosmo_py.plotting.tools import confidence_ellipse
from numcosmo_py.plotting.tools import set_rc_params_article
from numcosmo_py.plotting.getdist import mcat_to_mcsamples
from numcosmo_py.sampling.esmcmc import mcat_print_info

In [ ]:
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

nthreads = 4
nwalkers = 3000
burnin = 0
thin = 10
dim = 10
save_figs = True

Ncm.func_eval_set_max_threads(nthreads)
Ncm.func_eval_log_pool_stats()

In [ ]:
catalog_apes = run_funnel_mcmc(
    sampler="apes",
    nwalkers=nwalkers,
    dim=dim,
    ssize=7000000,
    robust=False,
    verbose=False,
    nthreads=4,
    over_smooth=2.0,
)
catalog_stretch = run_funnel_mcmc(
    sampler="stretch",
    nwalkers=nwalkers,
    dim=dim,
    ssize=7000000,
    verbose=False,
    nthreads=4,
)

In [ ]:
mcat_apes = Ncm.MSetCatalog.new_from_file_ro(catalog_apes, nwalkers * burnin)
mcat_stretch = Ncm.MSetCatalog.new_from_file_ro(catalog_stretch, nwalkers * burnin)

assert mcat_apes.nchains() == nwalkers
assert mcat_apes.nchains() == mcat_stretch.nchains()

In [ ]:
mcat_print_info(mcat_apes)
mcat_print_info(mcat_stretch)

In [ ]:
sample_apes, rows_apes, posterior_apes = mcat_to_mcsamples(
    mcat_apes,
    "APES",
    asinh_transform=list(range(1, 10)),
    thin=thin,
)
del mcat_apes

sample_stretch, rows_stretch, posterior_stretch = mcat_to_mcsamples(
    mcat_stretch,
    "STRETCH",
    asinh_transform=list(range(1, 10)),
    thin=thin,
)
del mcat_stretch

In [ ]:
print(
    sample_apes.getConvergeTests(
        what=("MeanVar", "GelmanRubin", "SplitTest", "CorrLengths"), feedback=True
    )
)
print(
    sample_stretch.getConvergeTests(
        what=("MeanVar", "GelmanRubin", "SplitTest", "CorrLengths"), feedback=True
    )
)

In [ ]:
set_rc_params_article(ncol=3)
g = plots.get_subplot_plotter(width_inch=plt.rcParams["figure.figsize"][0])
g.settings.linewidth = 0.01
g.triangle_plot(
    [sample_apes, sample_stretch],
    shaded=True,
    params=["nu", "asinh_x_1", "asinh_x_2", "asinh_x_3"],
)

if save_figs:
    plt.tight_layout()
    plt.savefig(f"funnel{dim}_corner.pdf")

In [ ]:
set_rc_params_article(ncol=3)
fig, ax = plt.subplots()

ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")

short_rows_apes = rows_apes[:nwalkers*600*3000,:]
short_rows_stretch = rows_stretch[:nwalkers*600*3000,:]

nframes = 600
b = np.unique(
    np.floor(np.linspace(0, short_rows_apes.shape[0] / nwalkers, nframes)).astype(int)
)
nframes = len(b)

scat_apes = ax.scatter(short_rows_apes[:, 0], short_rows_apes[:, 1], s=4, c="red")
scat_stretch = ax.scatter(rows_stretch[:, 0], rows_stretch[:, 1], s=4, c="blue")

ax.autoscale_view()


def animate(i):
    x_i = short_rows_apes[nwalkers * b[i] : nwalkers * b[i + 1], 0]
    y_i = short_rows_apes[nwalkers * b[i] : nwalkers * b[i + 1], 1]
    x2_i = rows_stretch[nwalkers * b[i] : nwalkers * b[i + 1], 0]
    y2_i = rows_stretch[nwalkers * b[i] : nwalkers * b[i + 1], 1]
    scat_apes.set_offsets(np.c_[x_i, y_i])
    scat_stretch.set_offsets(np.c_[x2_i, y2_i])


anim = FuncAnimation(fig, animate, interval=50, frames=nframes - 1)
pass

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
set_rc_params_article(ncol=3)

fig = plt.figure()
gs = fig.add_gridspec(2, hspace=0)
axs = gs.subplots(sharex=True)
fig.suptitle("Parameter evolution")

step = 1
alpha = 0.9
ps = 0.5
tmax = len(rows_apes) // nwalkers
t = np.arange(tmax, step=step)

for i in range(0, nwalkers, 150):
    x_1_apes = rows_apes[i::nwalkers, 0]
    x_1_stretch = rows_stretch[i::nwalkers, 0]
    x_2_apes = rows_apes[i::nwalkers, 1]
    x_2_stretch = rows_stretch[i::nwalkers, 1]

    axs[0].scatter(t, x_1_apes[0:tmax:step], s=ps, alpha=alpha, color="r")
    axs[0].scatter(t, x_1_stretch[0:tmax:step], s=ps, alpha=alpha, color="b")

    axs[1].scatter(t, x_2_apes[0:tmax:step], s=ps, alpha=alpha, color="r")
    axs[1].scatter(t, x_2_stretch[0:tmax:step], s=ps, alpha=alpha, color="b")

axs[0].set_ylabel("$x_1$")
axs[1].set_ylabel("$x_2$")
axs[1].set_xlabel("$t$")

if save_figs:
    plt.tight_layout()
    plt.savefig(f"funnel{dim}_param_evol.pdf")

pass

In [ ]:
kernel = Ncm.StatsDistKernelST.new(10, 1.0)
interp_vkde = Ncm.StatsDistVKDE.new(kernel, Ncm.StatsDistCV.NONE)
interp_kde = Ncm.StatsDistKDE.new(kernel, Ncm.StatsDistCV.NONE)

# interp_vkde.set_cov_type(Ncm.StatsDistKDECovType.ROBUST)
# interp_kde.set_cov_type(Ncm.StatsDistKDECovType.ROBUST)
interp_vkde.set_over_smooth(0.2)
interp_kde.set_over_smooth(0.2)

In [ ]:
max_n = len(rows_apes)
ssize = int(nwalkers / 2)

interp_vkde.reset()
interp_kde.reset()

for theta in rows_apes[-ssize:]:
    theta_v = Ncm.Vector.new_array(theta)
    interp_vkde.add_obs(theta_v)
    interp_kde.add_obs(theta_v)

m2lnL_v = Ncm.Vector.new_array(2.0 * posterior_apes[-ssize:])

interp_vkde.prepare_interp(m2lnL_v)
interp_kde.prepare_interp(m2lnL_v)

In [ ]:
set_rc_params_article(ncol=3)
fig, ax = plt.subplots(1, 1)

weights = np.array(interp_vkde.peek_weights().dup_array())
weights = weights / np.sum(weights)
lnweights = np.log(weights)

lnweights = lnweights - np.min(lnweights)
lnweights = lnweights / np.max(lnweights)

for i in range(interp_vkde.get_sample_size()):
    y_i, cov_i, n_i, w_i = interp_vkde.get_Ki(i)
    mu = y_i.dup_array()
    cov = np.array([[cov_i.get(i, j) for j in range(2)] for i in range(2)])
    cov = cov * 1.0
    confidence_ellipse(mu, cov, ax, edgecolor="red")

ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")
ax.autoscale_view()
plt.grid()

if save_figs:
    plt.tight_layout()
    plt.savefig(f"funnel{dim}_vkde_kernels.pdf")

In [ ]:
set_rc_params_article(ncol=3)
fig, ax = plt.subplots(1, 1)

for i in range(interp_kde.get_sample_size()):
    y_i, cov_i, n_i, w_i = interp_kde.get_Ki(i)
    mu = y_i.dup_array()
    cov = np.array([[cov_i.get(i, j) for j in range(2)] for i in range(2)])
    cov = cov * 1.0

    confidence_ellipse(mu, cov, ax, edgecolor="red")

ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")
ax.autoscale_view()
plt.grid()

if save_figs:
    plt.tight_layout()
    plt.savefig(f"funnel{dim}_kde_kernels.pdf")